根据训练计划对数据分拨

In [26]:
# 首先处理 meddialog—cn
import pandas as pd 

meddia_ch = pd.read_json('data\meddia_ch_single.jsonl',lines=True)
meddia_ch.shape

(46378, 3)

In [27]:
# 在含医学名词量最高的 2k 条中抽取一半作为测试集
top_2k = meddia_ch.iloc[:2000].copy()
meddia_ch = meddia_ch.iloc[2000:]
meddia_ch.shape

(44378, 3)

In [28]:
test_chn = top_2k.sample(1000, random_state=7)
non_sampled = top_2k.loc[top_2k.index.difference(test_chn.index)]
print((meddia_ch.shape, top_2k.shape, test_chn.shape, non_sampled.shape))

((44378, 3), (2000, 3), (1000, 3), (1000, 3))


In [29]:
test_chn.to_json('data\\test\\test_chn.jsonl', orient='records', lines=True)

In [30]:
meddia_ch = pd.concat([non_sampled,meddia_ch], axis=0)
meddia_ch.shape

(45378, 3)

In [31]:
#随机抽取 2k 条作为验证集
val_chn = meddia_ch.sample(2000, random_state=8)
val_chn.shape

(2000, 3)

In [ ]:
#剩下的作为微调训练集
meddia_ch = meddia_ch.loc[meddia_ch.index.difference(val_chn.index)]
meddia_ch.shape

(43378, 3)

In [34]:
val_chn.to_json('data\\val\\val_chn.jsonl', orient='records', lines=True)

In [35]:
meddia_ch.to_json('data\\train\\train_chn.jsonl', orient='records', lines=True)

In [36]:
#接着处理 hcm
hcm = pd.read_json('data\hcm_en.jsonl',lines=True)
val_en = hcm.sample(5000 , random_state=7)

In [39]:
hcm = hcm.loc[hcm.index.difference(val_en.index)]
hcm.shape

(99146, 2)

In [40]:
val_en.to_json('data\\val\\val_en.jsonl', orient='records', lines=True)
hcm.to_json('data\\train\\train_en.jsonl', orient='records', lines=True)

In [58]:
#最后处理 icliniq
icliniq_doc = pd.read_json('data\icl_en_real.jsonl',lines=True)
icliniq_mod = pd.read_json('data\icl_en_modl.jsonl',lines=True)

In [59]:
#选出做测试集的 1k 问题
test_en_real = icliniq_doc.sample(1000, random_state=8)
icliniq_doc = icliniq_doc.loc[icliniq_doc.index.difference(test_en_real.index)]
print((icliniq_doc.shape, test_en_real.shape))

test_en_modl = icliniq_mod.loc[test_en_real.index]
icliniq_mod = icliniq_mod.loc[icliniq_doc.index]
print((icliniq_mod.shape, test_en_modl.shape))

((5919, 2), (1000, 2))
((5919, 2), (1000, 2))


In [60]:
test_en_real.to_json('data\\test\\test_en_real.jsonl', orient='records', lines=True)
test_en_modl.to_json('data\\test\\test_en_modl.jsonl', orient='records', lines=True)

In [61]:
#风格多样化训练集： 70% 医生，30% 模型
train_en_diverse = icliniq_doc.sample(frac=0.7, random_state=7)
icliniq_mod = icliniq_mod.loc[icliniq_mod.index.difference(train_en_diverse.index)]
print((train_en_diverse.shape, icliniq_mod.shape))

((4143, 2), (1776, 2))


In [62]:
#添加模型或医生回答的 identifier
train_en_diverse['type'] = "real"
icliniq_mod['type'] = "model"

In [63]:
train_en_diverse = pd.concat([train_en_diverse, icliniq_mod], axis=0)
train_en_diverse = train_en_diverse.sample(frac=1) #shuffle
train_en_diverse.shape

(5919, 3)

In [64]:
train_en_diverse.to_json('data\\train\\train_en_diverse.jsonl', orient='records', lines=True)